In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import os
os.chdir('/content/drive/MyDrive/딥러닝개론/Final Project')

# data

In [ ]:
rev_data = pd.read_csv('/content/drive/MyDrive/딥러닝개론/Final Project/data/rev_data_final.csv', index_col=0)
rev_data

,review,score,label
0,친절하시고 깔끔하고 좋았습니다,5,1
1,조용하고 고기도 굿,5,1
2,"갈비탕과 냉면, 육회비빔밥이 맛있습니다.",4,1
3,대체적으로 만족하나\n와인의 구성이 살짝 아쉬움,4,1
4,고기도 맛있고 서비스는 더 최고입니다~,5,1
...,...,...,...
8009,가짜원조... 줄서는사람 뭐지???,1,0
8010,짝퉁 여긴 원조랑 아무 관계 직원 연관되게 없는 그냥 이름만 .. 가짜입니다 속지 마세요,1,0
8011,가성비 별로 비싸기만함,1,0
8012,별 한개주기도 아까움. 별을 줘야 등록돼서 평가함! 나 같은 사람 또 안생겼으면 좋...,1,0


# KoGPT-2

## 이중분류 without shopping_rev_final.csv

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 13.4 MB/s 
     |████████████████████████████████| 182 kB 78.3 MB/s 
     |████████████████████████████████| 7.6 MB 57.5 MB/s 


In [ ]:
import transformers
transformers.__version__

'4.25.1'

In [ ]:
import pandas as pd
import numpy as np
import urllib.request
import os
from tqdm import tqdm
import tensorflow as tf
from transformers import AutoTokenizer, TFGPT2Model
from tensorflow.keras.preprocessing.sequence import pad_sequences

### train & test split

In [ ]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(rev_data, test_size=0.2, random_state=0, stratify=rev_data['label'])

In [ ]:
print('train_data shape is:', train_data.shape)
print('test_data shape is:', test_data.shape)

train_data shape is: (7144, 3)
test_data shape is: (1787, 3)


### tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', pad_token='<pad>')

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


#### tokenizer 확인

In [ ]:
print(tokenizer.encode(train_data.iloc[0, 0]))

[9525, 7496, 8153, 7172, 7182, 9028, 7799, 7692, 26490, 8084, 739]


In [ ]:
print(tokenizer.tokenize(train_data.iloc[0, 0]))

['▁존', '맛', '입', '니', '다', '▁가', '성', '비', '▁좋고', '요', '▁']


In [ ]:
tokenizer.decode(tokenizer.encode(train_data.iloc[0, 0]))

'존맛입니다 가성비 좋고요 '

In [ ]:
for elem in tokenizer.encode(train_data.iloc[0, 0]):
  print(tokenizer.decode(elem))

존
맛
입
니
다
가
성
비
좋고
요



In [ ]:
encoded_result = tokenizer.encode(train_data.iloc[0, 0], max_length=128, pad_to_max_length=True, truncation=True)
print(encoded_result)
print('길이 :', len(encoded_result))

[9525, 7496, 8153, 7172, 7182, 9028, 7799, 7692, 26490, 8084, 739, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
길이 : 128


/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


#### tokenizer 설정

In [ ]:
max_seq_len = 128

In [ ]:
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer):

    input_ids, data_labels = [], []
    
    for example, label in tqdm(zip(examples, labels), total=len(examples)):

        bos_token = [tokenizer.bos_token]
        eos_token = [tokenizer.eos_token]
        tokens = bos_token + tokenizer.tokenize(example) + eos_token
        input_id = tokenizer.convert_tokens_to_ids(tokens)
        input_id = pad_sequences([input_id], maxlen=max_seq_len, value=tokenizer.pad_token_id, padding='post')[0]

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        input_ids.append(input_id)
        data_labels.append(label)

    input_ids = np.array(input_ids, dtype=int)
    data_labels = np.asarray(data_labels, dtype=np.int32)

    return input_ids, data_labels

In [ ]:
train_X, train_y = convert_examples_to_features(train_data['review'], train_data['label'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 7144/7144 [00:02<00:00, 2710.05it/s]


In [ ]:
test_X, test_y = convert_examples_to_features(test_data['review'], test_data['label'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 1787/1787 [00:00<00:00, 2314.58it/s]


In [ ]:
# 최대 길이: 128
input_id = train_X[0]
label = train_y[0]

print('단어에 대한 정수 인코딩 :',input_id)
print('각 인코딩의 길이 :', len(input_id))
print('정수 인코딩 복원 :',tokenizer.decode(input_id))
print('레이블 :',label)

단어에 대한 정수 인코딩 : [    1  9525  7496  8153  7172  7182  9028  7799  7692 26490  8084   739
     1     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3]
각 인코딩의 길이 : 128
정수 인코딩 복원 : </s> 존맛입니다 가성비 좋고요 </s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pa

### model

In [ ]:
model = TFGPT2Model.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

Downloading:   0%|          | 0.00/513M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2Model: ['transformer.h.9.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'lm_head.weight', 'transformer.h.0.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.5.attn.masked_bias']
- This IS expected if you are initializing TFGPT2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All t

In [ ]:
input_ids_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)
outputs = model([input_ids_layer])

In [ ]:
print(outputs)

TFBaseModelOutputWithPastAndCrossAttentions(last_hidden_state=<KerasTensor: shape=(None, 128, 768) dtype=float32 (created by layer 'tfgpt2_model')>, past_key_values=(<KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')

In [ ]:
print(outputs[0])

KerasTensor(type_spec=TensorSpec(shape=(None, 128, 768), dtype=tf.float32, name=None), name='tfgpt2_model/transformer/Reshape_2:0', description="created by layer 'tfgpt2_model'")


In [ ]:
class TFGPT2ForSequenceClassification(tf.keras.Model):
    def __init__(self, model_name):
        super(TFGPT2ForSequenceClassification, self).__init__()
        self.gpt = TFGPT2Model.from_pretrained(model_name, from_pt=True)
        self.dropout = tf.keras.layers.Dropout(0.2)
        self.classifier = tf.keras.layers.Dense(1,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                activation='sigmoid',
                                                name='classifier')

    def call(self, inputs):
        outputs = self.gpt(input_ids=inputs)
        cls_token = outputs[0][:, -1]
        cls_token = self.dropout(cls_token)
        prediction = self.classifier(cls_token)

        return prediction

#### TPU 사용시

In [ ]:
# TPU 작동을 위한 코드
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

In [ ]:
strategy = tf.distribute.experimental.TPUStrategy(resolver)

In [ ]:
with strategy.scope():
  model = TFGPT2ForSequenceClassification("skt/kogpt2-base-v2")
  optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
  loss = tf.keras.losses.BinaryCrossentropy()
  model.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy'])

#### GPU 사용시

In [ ]:
model = TFGPT2ForSequenceClassification("skt/kogpt2-base-v2")
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.BinaryCrossentropy()
model.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2Model: ['transformer.h.9.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'lm_head.weight', 'transformer.h.0.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.5.attn.masked_bias']
- This IS expected if you are initializing TFGPT2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All t

#### -

In [ ]:
model.fit(train_X, train_y, epochs=20, batch_size=32, validation_split=0.2)

Epoch 1/20
179/179 [==============================] - 184s 947ms/step - loss: 0.6315 - accuracy: 0.6975 - val_loss: 0.3785 - val_accuracy: 0.8348
Epoch 2/20
179/179 [==============================] - 167s 934ms/step - loss: 0.3272 - accuracy: 0.8663 - val_loss: 0.3404 - val_accuracy: 0.8789
Epoch 3/20
179/179 [==============================] - 167s 936ms/step - loss: 0.2268 - accuracy: 0.9108 - val_loss: 0.3708 - val_accuracy: 0.8572
Epoch 4/20
179/179 [==============================] - 167s 935ms/step - loss: 0.1576 - accuracy: 0.9410 - val_loss: 0.4034 - val_accuracy: 0.8558
Epoch 5/20
179/179 [==============================] - 167s 934ms/step - loss: 0.1051 - accuracy: 0.9617 - val_loss: 0.5868 - val_accuracy: 0.8509
Epoch 6/20
179/179 [==============================] - 167s 933ms/step - loss: 0.0723 - accuracy: 0.9731 - val_loss: 0.5043 - val_accuracy: 0.8817
Epoch 7/20
179/179 [==============================] - 167s 935ms/step - loss: 0.0671 - accuracy: 0.9773 - val_loss: 0.6225 -

In [ ]:
results = model.evaluate(test_X, test_y, batch_size=1024)
print("test loss, test acc: ", results)

2/2 [==============================] - 16s 6s/step - loss: 0.7371 - accuracy: 0.8674
test loss, test acc:  [0.7370892763137817, 0.8673754930496216]


In [ ]:
model.save_weights('/content/drive/MyDrive/딥러닝개론/Final Project/koGPT-2_shopx.h5')

### test

In [ ]:
model = TFGPT2Model.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

In [ ]:
model = TFGPT2ForSequenceClassification("skt/kogpt2-base-v2")
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.BinaryCrossentropy()
model.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2Model: ['transformer.h.11.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'lm_head.weight', 'transformer.h.8.attn.masked_bias', 'transformer.h.6.attn.masked_bias']
- This IS expected if you are initializing TFGPT2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All t

In [ ]:
model.fit(train_X, train_y, epochs=1, batch_size=32, validation_split=0.2)

179/179 [==============================] - 180s 923ms/step - loss: 0.7243 - accuracy: 0.6980 - val_loss: 0.3731 - val_accuracy: 0.8502


In [ ]:
model.load_weights('/content/drive/MyDrive/딥러닝개론/Final Project/koGPT-2_shopx.h5')

In [ ]:
model.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy'])

In [ ]:
results = model.evaluate(test_X, test_y, batch_size=1024)
print("test loss, test acc: ", results)

In [ ]:
def sentiment_predict(new_sentence):

  bos_token = [tokenizer.bos_token]
  eos_token = [tokenizer.eos_token]
  tokens = bos_token + tokenizer.tokenize(new_sentence) + eos_token
  input_id = tokenizer.convert_tokens_to_ids(tokens)
  input_id = pad_sequences([input_id], maxlen=max_seq_len, value=tokenizer.pad_token_id, padding='post')[0]
  input_id = np.array([input_id])
  score = model.predict(input_id)[0][0]

  if(score > 0.5):
    print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
  else:
    print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))

#### 시험용

In [ ]:
sentiment_predict('너무 맛있는데 직원이 좀 불친절해요.')

1/1 [==============================] - 3s 3s/step
58.47% 확률로 긍정 리뷰입니다.



In [ ]:
sentiment_predict('음식 맛도 좋고 데이트하기 좋아요요')

1/1 [==============================] - 0s 23ms/step
99.99% 확률로 긍정 리뷰입니다.



In [ ]:
sentiment_predict('존맛탱탱')

1/1 [==============================] - 0s 22ms/step
100.00% 확률로 긍정 리뷰입니다.



In [ ]:
sentiment_predict('개노맛')

1/1 [==============================] - 0s 23ms/step
100.00% 확률로 긍정 리뷰입니다.



In [ ]:
sentiment_predict('너무 별로에요 다신 안갈듯')

1/1 [==============================] - 0s 22ms/step
100.00% 확률로 부정 리뷰입니다.



In [ ]:
sentiment_predict('걍그래요')

1/1 [==============================] - 0s 23ms/step
72.08% 확률로 긍정 리뷰입니다.



In [ ]:
sentiment_predict('맛없어요')

1/1 [==============================] - 0s 24ms/step
100.00% 확률로 부정 리뷰입니다.



In [ ]:
sentiment_predict('JMT')

1/1 [==============================] - 0s 27ms/step
99.94% 확률로 긍정 리뷰입니다.



In [ ]:
sentiment_predict('존맛탱구리')

1/1 [==============================] - 0s 22ms/step
100.00% 확률로 긍정 리뷰입니다.



In [ ]:
sentiment_predict('노맛')

1/1 [==============================] - 0s 29ms/step
75.24% 확률로 부정 리뷰입니다.

